# Kaggle MNIST Challenge

Using a Deep Residual Neural Network to classify digits for the Kaggle MNIST challenge.

#### Set seed for reproducibility

In [ ]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Input, Add, Dense, Dropout, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, LearningRateScheduler

from sklearn.model_selection import train_test_split

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

#### Load data

In [ ]:
n_classes = 10

raw_data = np.loadtxt('kaggle/datasets/mnist/train.csv', skiprows=1, dtype='int', delimiter=',')
X_train, X_val, y_train, y_val = train_test_split(
    raw_data[:,1:], raw_data[:,0], test_size = 0.1)

X_train = X_train.reshape(-1, 28, 28, 1)
X_val = X_val.reshape(-1, 28, 28, 1)

X_train = X_train.astype("float32")/255.
X_val = X_val.astype("float32")/255.

y_train = keras.utils.to_categorical(y_train, n_classes)
y_val = keras.utils.to_categorical(y_val, n_classes)


#### Define Custom Activation Function

In [ ]:
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

def relus(Z):
    e_param = 0.10318
    pi = K.variable((3.14))
    m = e_param + (K.sigmoid(K.sin(Z)) - K.sigmoid(K.cos(Z)) * K.exp(K.sqrt(pi)))
    A = K.maximum(m, Z)
    return A

get_custom_objects().update({'ReLU_s': Activation(relus)})

#### ResNet Identity Block

In [ ]:
def identity_block(X, main_path_shape, filters, stage, block):
    """
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    main_path_shape -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    # Kernel Size is (1, 1), or just 1
    # Default strides is (1, 1)
    X = Conv2D(F1, kernel_size = 1, padding = 'valid', name = conv_name_base + '2a', kernel_initializer = 'he_uniform')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('ReLU_s')(X)
    
    # Second component of main path
    # Kernel Size is (main_path_shape, main_path_shape)
    # Default strides is (1, 1)
    X = Conv2D(F2, kernel_size = main_path_shape, padding = 'same', name = conv_name_base + '2b', kernel_initializer = 'he_uniform')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('ReLU_s')(X)

    # Third component of main path
    # Default strides is (1, 1)
    X = Conv2D(F3, kernel_size = 1, padding = 'valid', name = conv_name_base + '2c', kernel_initializer = 'he_uniform')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut, X])
    X = Activation('ReLU_s')(X)
    
    return X

#### ResNet Convolutional Block

In [ ]:
def convolutional_block(X, main_path_shape, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    main_path_shape -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, kernel_size = 1, strides = s, name = conv_name_base + '2a', kernel_initializer = 'he_uniform')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('ReLU_s')(X)

    # Second component of main path
    X = Conv2D(F2, kernel_size = main_path_shape, padding = 'same', name = conv_name_base + '2b', kernel_initializer = 'he_uniform')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('ReLU_s')(X)

    # Third component of main path
    X = Conv2D(F3, kernel_size = 1, padding = 'valid', name = conv_name_base + '2c', kernel_initializer = 'he_uniform')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(F3, kernel_size = 1, strides = s, padding = 'valid', name = conv_name_base + '1', kernel_initializer = 'he_uniform')(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut, X])
    X = Activation('ReLU_s')(X)
    
    return X

#### Design Neural Network architecture

In [ ]:
def ResNetE(input_shape = (28, 28, 1)):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK -> CONVBLOCK -> IDBLOCK
    -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FC -> SOFTMAX

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(8, kernel_size = 3, name = 'conv1', kernel_initializer = 'he_uniform')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('ReLU_s')(X)
    X = MaxPooling2D(pool_size = 3)(X)
    
    # Stage 2
    X = convolutional_block(X, main_path_shape = 3, filters = [16, 16, 24], stage = 2, block = 'a', s = 1)
    X = identity_block(X, 3, [16, 16, 24], stage = 2, block = 'b')
    
    # Stage 3
    X = convolutional_block(X, main_path_shape = 3, filters = [24, 24, 32], stage = 3, block = 'a', s = 2)
    X = identity_block(X, 3, [24, 24, 32], stage = 3, block = 'b')

    # Stage 4
    X = convolutional_block(X, main_path_shape = 3, filters = [32, 32, 48], stage = 4, block = 'a', s = 2)
    X = identity_block(X, 3, [32, 32, 48], stage = 4, block='b')
    X = identity_block(X, 3, [32, 32, 48], stage = 4, block='c')
    X = identity_block(X, 3, [32, 32, 48], stage = 4, block='d')
    X = identity_block(X, 3, [32, 32, 48], stage = 4, block='e')

    # Stage 5
    X = convolutional_block(X, main_path_shape = 3, filters = [48, 48, 64], stage = 5, block = 'a', s = 2)
    X = identity_block(X, 3, [48, 48, 64], stage = 5, block = 'b')
    X = identity_block(X, 3, [48, 48, 64], stage = 5, block = 'c')
    X = identity_block(X, 3, [48, 48, 64], stage = 5, block = 'd')
    X = identity_block(X, 3, [48, 48, 64], stage = 5, block = 'e')

    # AVGPOOL
    X = AveragePooling2D(pool_size = 1, name = 'avg_pool')(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(512, activation='ReLU_s', name = 'fc1' + str(512), kernel_initializer = 'he_uniform')(X)
    X = Dense(n_classes, activation='softmax', name = 'fc' + str(n_classes), kernel_initializer = 'glorot_uniform')(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name = 'ResNetE')

    return model

In [ ]:
model = ResNetE()
model.summary()

#### Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

test_datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

train_generator = train_datagen.flow(X_train, y_train, batch_size = 24)

validation_generator = test_datagen.flow(X_val, y_val, batch_size = 24)

#### Learning Rate Scheduler

In [ ]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

#### Configure model

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#### TensorBoard

In [ ]:
tensorboard = TensorBoard("../logs/resnet-mnist-ReLUs-60-epochs-batch-32-ResNet-E-data-augmentation")

#### Train!

In [ ]:
hist = model.fit_generator(train_generator,
                           steps_per_epoch = 5000,
                           epochs = 60,
                           verbose = 1,
                           validation_data = validation_generator,
                           validation_steps = 3500,
                           callbacks = [annealer, tensorboard])

# hist = model.fit(X_train, y_train, batch_size = 32,
#                            epochs = 40,
#                            verbose = 1,
#                            validation_split = 0.1,
#                            callbacks=[annealer, tensorboard])

#### Test Final Accuracy

In [ ]:
final_loss, final_acc = model.evaluate(X_val, y_val, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

#### Save Submission

In [ ]:
# X_test = np.loadtxt('kaggle/datasets/mnist/test.csv', skiprows=1, dtype='int', delimiter=',')
# X_test = X_test.reshape(28000, 28, 28, 1).astype('float32') / 255.

# predictions = model.predict(X_test, verbose = 2)
# predictions = np.argmax(predictions, axis = 1)

# pd.DataFrame({"ImageId": list(range(1, len(predictions) + 1)), "Label": predictions}).to_csv('kaggle/results/mnist/submission-ReLUs-30-epochs-batch-64-ResNet-E-data-augmentation.csv', index = False, header = True)